In [1]:
from selenium import webdriver

In [2]:
import datetime

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.capabilities['chrome']['chromedriverVersion'].split()[0]

'124.0.6367.91'

In [5]:
# chrome_driver = '\\Users\\91700\\Downloads\\chromedriver-win64\\chromedriver'
# may be required sometime

In [ ]:
# Import necessary libraries
from selenium import webdriver  # For automating web browser interactions
from selenium.webdriver.common.by import By  # For locating elements on web pages
from bs4 import BeautifulSoup  # For parsing HTML and XML documents
import time  # For adding delays in the script
import warnings  # For managing warning messages
import pandas as pd  # For working with data in tabular format
from tqdm import tqdm  # For displaying progress bars
import csv  # For reading and writing CSV files
import random  # For generating random numbers
import datetime  # For working with dates and times

# Ignore any warning messages
warnings.filterwarnings("ignore")

# Read the Excel file containing company names into a DataFrame
df = pd.read_excel("Company_names.xlsx")

# Display the first 10 rows of the DataFrame
df.head(10)

# Read the LinkedIn authentication information (li_at cookie) from a text file
with open("config.txt", 'r', encoding="utf-8") as f1:
    lines = f1.readlines()
li_at = lines[0]

# Define the LinkedIn cookie
cookies = {
    'name': 'li_at',
    'value': li_at,
    'domain': '.linkedin.com',
}

# Initialize a Chrome WebDriver
driver = webdriver.Chrome()

# Navigate to the LinkedIn login page
driver.get("https://www.linkedin.com")
time.sleep(random.randint(60,65))
# Maximize the browser window
driver.maximize_window()

# Add the LinkedIn cookie to the WebDriver session
driver.add_cookie(cookies)

# Create an empty DataFrame to store company information
transfer_sheet = pd.DataFrame(columns=["Company Name", "About"])

# Initialize a counter variable
c = 0

# Iterate through each company name in the DataFrame
for cn in df["Name"]:
    # Construct the search URL for the company on LinkedIn
    search_url = f"https://www.linkedin.com/search/results/companies/?keywords={cn}&origin=SWITCH_SEARCH_VERTICAL&sid=7bb"
    
    # Navigate to the search URL
    driver.get(search_url)
    
    # Add a random delay to simulate human-like behavior
    time.sleep(random.randint(2, 5))
    
    try:
        # Find the first search result link for the company
        jc = driver.find_elements(By.CSS_SELECTOR, ".reusable-search__result-container a.app-aware-link")
        elems = list(jc)
        lnks = elems[0].get_attribute('href') 
        
        # Navigate to the company page
        driver.get(lnks)
        
        # Add a random delay
        time.sleep(random.randint(3, 7))
        
        try:
            # Navigate to the "About" section of the company page
            a = ""
            for i in driver.find_elements(By.CSS_SELECTOR, ".org-page-navigation__item a"):
                x1 = i.get_attribute('href')
                if 'about' in str(x1):
                    a = x1
            driver.get(a)
        except:
            print("Link not Found")
        
        # Add a random delay
        time.sleep(10)
        
        # Extract information about the company
        b = ""
        elem2 = driver.find_elements(By.CSS_SELECTOR, ".overflow-hidden dd")
        for k in elem2:
            b += " " + k.text
        
        # Store the extracted information in the DataFrame
        transfer_sheet.loc[c] = [cn, b]
        c += 1
        
        # Write the DataFrame to an Excel file
        transfer_sheet.to_excel(r"C:\Users\91700\Desktop\DT\Scrapper\data.xlsx", engine='xlsxwriter')

        
        # Add a random delay
        time.sleep(random.randint(1, 5))
    
    except:
        # If company information cannot be retrieved, mark it as "No results" in the DataFrame
        transfer_sheet.loc[c] = [cn, "No results"]
        transfer_sheet.to_excel(r"C:\Users\91700\Desktop\DT\Scrapper\data.xlsx", engine='xlsxwriter')


# End of the loop

# Display completion message
print("Scraping completed!")
# Close the WebDriver
driver.quit()